---
description: Core Classes for hwpapi
output-file: core.html
title: core

---

In [4]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
#| default_exp core

In [6]:
#| hide
from nbdev.showdoc import *

In [7]:
#| hide
#| export
from contextlib import contextmanager
from pathlib import Path
import numbers
import hwpapi.constants as const
from fastcore.basics import patch

In [8]:
#| hide
#| export
from hwpapi.actions import _Action, _Actions
from hwpapi.dataclasses import CharShape, ParaShape
from hwpapi.functions import (
    check_dll,
    get_hwp_objects,
    dispatch,
    get_absolute_path,
    get_charshape_pset,
    get_parashape_pset,
    get_rgb_tuple,
    get_value,
    mili2unit,
    point2unit,
    set_charshape_pset,
    set_parashape_pset,
    set_pset,
    unit2mili,
    unit2point,
)

In [9]:
#| export
class Engine:
    """
    Engine class to encapsulate the Hancom Office Hwp object.

    This class provides an interface to interact with the Hancom Office Hwp application,
    facilitating actions and operations within the Hwp environment.

    Parameters
    ----------
    hwp_object : object, optional
        The Hwp object to be encapsulated by the Engine. If not provided, 
        it defaults to creating a new Hwp object using "HWPFrame.HwpObject".

    Attributes
    ----------
    impl : object
        The implementation of the Hwp object.

    Methods
    -------
    name()
        Returns the name (CLSID) of the Hwp object.

    Examples
    --------
    >>> engine = Engine()
    >>> print(engine.name)
    """

    def __init__(self, hwp_object=None):
        """
        Initializes the Engine with a Hancom Hwp object.

        If `hwp_object` is not provided, a new Hwp object is created. If the creation fails, `impl` is set to None.

        Parameters
        ----------
        hwp_object : object, optional
            The Hwp object to be encapsulated by the Engine. Defaults to "HWPFrame.HwpObject".
        """
        try:
            if not hwp_object:
                hwp_object = "HWPFrame.HwpObject"
            self.impl = dispatch(hwp_object)
        except Exception as e:
            print(f"Failed to initialize Hwp object: {e}")
            self.impl = None

    @property
    def name(self):
        """
        Returns the name (CLSID) of the Hwp object.

        Returns
        -------
        str
            The CLSID of the Hwp object. Returns None if the object is not initialized.
        """
        return self.impl.CLSID if self.impl else None

    def __repr__(self):
        """
        Returns the string representation of the Engine object.

        Returns
        -------
        str
            String representation of the Engine. If the engine is uninitialized, it indicates 'Uninitialized'.
        """
        engine_name = self.name if self.name else "Uninitialized"
        return f"<Engine {engine_name}>"


In [10]:
#| export
class Engines:
    """
    A collection manager for multiple Engine instances.

    This class manages multiple Engine instances, providing methods to access and iterate over them.
    It is useful for handling multiple Hancom Office Hwp objects.

    Parameters
    ----------
    dll_path : str, optional
        Path to a DLL file, if required for initialization.

    Attributes
    ----------
    active : Engine or None
        The currently active Engine instance.
    engines : list of Engine
        List of Engine instances managed by this collection.

    Methods
    -------
    add(engine)
        Adds a new Engine instance to the collection.
    count()
        Returns the number of Engine instances in the collection.

    Examples
    --------
    >>> engines = Engines()
    >>> engines.add(Engine())
    >>> print(engines.count)
    >>> for engine in engines:
    ...     print(engine)

    Notes
    -----
    The `Engines` class initializes by creating Engine instances for each object retrieved
    from `get_hwp_objects()` function. It also checks for the required DLLs if `dll_path` is provided.
    """

    def __init__(self, dll_path=None):
        """
        Initializes the Engines collection with available Hwp objects.

        Parameters
        ----------
        dll_path : str, optional
            Path to a DLL file, if required for initialization.
        """
        self.active = None
        self.engines = [Engine(hwp_object) for hwp_object in get_hwp_objects()]
        check_dll(dll_path)

    def add(self, engine):
        """
        Adds a new Engine instance to the collection.

        Parameters
        ----------
        engine : Engine
            The Engine instance to be added to the collection.
        """
        self.engines.append(engine)

    @property
    def count(self):
        """
        Returns the number of Engine instances in the collection.

        Returns
        -------
        int
            The number of Engine instances.
        """
        return len(self)

    def __call__(self, ind):
        """
        Returns an Engine instance from the collection based on its index.

        Parameters
        ----------
        ind : int
            The index of the Engine instance to retrieve.

        Returns
        -------
        Engine
            The Engine instance at the specified index.
        """
        if isinstance(ind, numbers.Number):
            return self.engines[ind]

    def __getitem__(self, ind):
        """
        Returns an Engine instance from the collection based on its index.

        Parameters
        ----------
        ind : int
            The index of the Engine instance to retrieve.

        Returns
        -------
        Engine
            The Engine instance at the specified index.
        """
        if isinstance(ind, numbers.Number):
            return self.engines[ind]

    def __len__(self):
        """
        Returns the number of Engine instances in the collection.

        Returns
        -------
        int
            The number of Engine instances.
        """
        return len(self.engines)

    def __iter__(self):
        """
        Yields each Engine instance in the collection.

        Yields
        ------
        Engine
            Each Engine instance in the collection.
        """
        for engine in self.engines:
            yield engine

    def __repr__(self):
        """
        Returns the string representation of the Engines collection.

        Returns
        -------
        str
            String representation of the Engines collection.
        """
        return f"<HWP Engines activated: {len(self.engines)}>"


In [11]:
#| export

class Apps:
    """
    A collection of all `app <App>` objects.

    Attributes
    ----------
    _apps : list
        A list containing instances of `App`.

    Methods
    -------
    add(**kwargs)
        Creates a new App and adds it to the collection.
    count()
        Returns the number of apps in the collection.
    cleanup()
        [Method description needed]
    """

    def __init__(self):
        """
        Initializes the `Apps` collection by creating `App` instances for each `engine` in `Engines`.
        """
        self._apps = [App(engine=engine) for engine in Engines()]

    def add(self, **kwargs):
        """
        Creates a new App. The new App becomes the active one.

        Returns
        -------
        App
            The newly created App object.
        """
        app = App(engine=Engine())
        self._apps.append(app)
        return app  

    def __call__(self, i):
        """
        Allows the `Apps` instance to be called like a function, returning the app at the specified index.

        Parameters
        ----------
        i : int
            The index of the app to retrieve.

        Returns
        -------
        App
            The app at the specified index.
        """
        return self[i]

    def __repr__(self):
        """
        Returns a string representation of the `Apps` instance.

        Returns
        -------
        str
            A string representation of the `Apps` instance.
        """
        return "{}({})".format(
            getattr(self.__class__, "_name", self.__class__.__name__), repr(list(self))
        )

    def __getitem__(self, item):
        """
        Returns the app at the specified index.

        Parameters
        ----------
        item : int
            The index of the app.

        Returns
        -------
        App
            The app at the specified index.
        """
        return self._apps[item]

    def __len__(self):
        """
        Returns the number of apps in the collection.

        Returns
        -------
        int
            The number of apps.
        """
        return len(self._apps)

    @property
    def count(self):
        """
        Returns the number of apps in the collection.

        .. versionadded:: 0.9.0

        Returns
        -------
        int
            The number of apps.
        """
        return len(self)

    def cleanup(self):
        """
        [Method description needed]

        [Describe what the method does and any side effects]
        """
        

    def __iter__(self):
        """
        Allows iteration over the collection of apps.

        Yields
        ------
        App
            The next app in the collection.
        """
        for app in self._apps:
            yield app


In [12]:
#| export 
#| hide

engines = Engines()

In [13]:
# | export
class App:
    """
    `App` 클래스는 한컴오피스의 한/글 프로그램과 상호작용하기 위한 인터페이스를 제공합니다.

    이 클래스는 한/글 프로그램의 COM 객체와의 연동을 관리하고, 해당 객체에 대한 작업을 수행하는 메서드들을 포함합니다.

    Attributes
    ----------
    engine : Engine, optional
        사용할 엔진 객체입니다.
    is_visible : bool, optional
        한/글 프로그램의 가시성을 설정합니다.
    dll_path : str, optional
        사용할 DLL 파일의 경로입니다.

    Methods
    -------
    __init__(engine=None, is_visible=True, dll_path=None)
        `App` 클래스의 인스턴스를 초기화합니다.
    api()
        현재 사용 중인 엔진의 구현체를 반환합니다.
    __str__()
        `App` 인스턴스의 문자열 표현을 반환합니다.
    """

    def __init__(
        self,
        new_app=False,
        is_visible=True,
        dll_path=None,
        engine=None,
    ):
        """
        `App` 클래스의 인스턴스를 초기화합니다.

        Parameters
        ----------
        new_app: bool, optional
            별도 앱을 열지를 결정합니다.
        is_visible : bool, optional
            한/글 프로그램의 가시성 설정. 기본값은 True입니다.
        dll_path : str, optional
            사용할 DLL 파일의 경로. 기본값은 None입니다.
        engine : Engine, optional
            한/글 프로그램과의 상호작용을 위한 엔진. 기본값이 없을 경우, 새로운 엔진 인스턴스가 생성됩니다.
        Notes
        -----
        engine이 제공되지 않은 경우, `Engines`를 통해 엔진을 생성하거나 선택합니다.
        """
        self._load(new_app=new_app, dll_path=dll_path, engine=engine)
        self.actions = _Actions(self)
        self.parameters = self.api.HParameterSet
        self.set_visible(is_visible)

    def _load(self, new_app, engine, dll_path):
        if new_app:
            engine = Engine()
        if not engine:
            engines = Engines()
            engine = engines[-1] if len(engines) > 0 else Engine()
        self.engine = engine
        check_dll(dll_path)
        self.api.RegisterModule("FilePathCheckDLL", "FilePathCheckerModule")
        

    @property
    def api(self):
        """
        현재 사용 중인 엔진의 구현체를 반환합니다.

        Returns
        -------
        object
            엔진의 구현체.
        """
        return self.engine.impl

    def __str__(self):
        """
        `App` 인스턴스의 문자열 표현을 반환합니다.

        Returns
        -------
        str
            `App` 인스턴스를 설명하는 문자열.
        """
        return f"<Hwp App: {self.get_filepath()}>"

    __repr__ = __str__

    # add attribution for patched functions for autocompletion
    close = 'patch' 
    create_action = 'patch'
    create_parameterset = 'patch'
    engine = 'patch'
    find_text = 'patch'
    get_charshape = 'patch'
    get_filepath = 'patch'
    get_font_list = 'patch'
    get_hwnd = 'patch'
    get_parashape = 'patch'
    get_selected_text = 'patch'
    get_text = 'patch'
    insert_file = 'patch'
    insert_picture = 'patch'
    insert_text = 'patch'
    move = 'patch'
    open = 'patch'
    quit = 'patch'
    reload = 'patch'
    replace_all = 'patch'
    save = 'patch'
    save_block = 'patch'
    scan = 'patch'
    select_text = 'patch'
    set_cell_border = 'patch'
    set_cell_color = 'patch'
    set_charshape = 'patch'
    set_parashape = 'patch'
    set_visible = 'patch'
    setup_page = 'patch'


In [14]:
#| export

@patch
def reload(app: App, new_app=False, dll_path=None):
    """
    Reloads the `App` instance with a new HWPFrame.HwpObject and resets its visibility and DLL path.

    This function is designed to reinitialize the `app` object's API with a new instance of the HWPFrame.HwpObject.
    It also checks and registers the specified DLL file if provided.

    Parameters
    ----------
    app : App
        The `App` instance to be reloaded.
    dll_path : str, optional
        The path to the DLL file to be checked and registered. If None, no DLL registration is performed.

    Notes
    -----
    The `reload` function is useful in scenarios where the state of the HWPFrame.HwpObject needs to be reset
    or when changing the DLL being used.

    Examples
    --------
    >>> app = App()
    >>> reload(app, dll_path="path/to/dll")
    """
    app._load(new_app=new_app, dll_path=dll_path)


In [15]:
#| export
@patch
def set_visible(app: App, is_visible=True, window_i=0):
    """
    Sets the visibility of the Hancom Office Hwp program window.

    This function is used to show or hide the window of the Hancom Office Hwp program based on the `is_visible` parameter.
    The `window_i` parameter specifies the index of the window to be shown or hidden.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    is_visible : bool, optional
        A boolean value indicating the visibility of the program window. 
        If True, the window is made visible. If False, it is hidden. Default is True.
    window_i : int, optional
        The index of the window to be modified. Default is 0.

    Examples
    --------
    >>> app = App()
    >>> set_visible(app, is_visible=True, window_i=0)
    >>> set_visible(app, is_visible=False, window_i=1)
    """
    app.api.XHwpWindows.Item(window_i).Visible = is_visible


In [16]:
#| export
@patch
def get_filepath(app: App):
    """
    Retrieves the file path of the currently open Hancom Office Hwp document.

    This function accesses the active document in the Hancom Office Hwp program linked to the `App` instance
    and returns its full file path.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.

    Returns
    -------
    str
        The full file path of the currently active Hancom Office Hwp document.

    Examples
    --------
    >>> app = App()
    >>> filepath = get_filepath(app)
    >>> print(filepath)
    """
    doc = app.api.XHwpDocuments.Active_XHwpDocument
    return doc.FullName


In [17]:
#| export
@patch
def create_action(app: App, action_key: str, pset_key=None):
    """
    Creates and returns an instance of the `_Action` class.

    This function generates a new `_Action` instance associated with the given `app` and `action_key`. 
    The `action_key` specifies the type of action to be created for the application.

    Parameters
    ----------
    app : App
        The application object for which the action is being created.
    action_key : str
        The key representing the specific action to be created.

    Returns
    -------
    _Action
        An instance of the `_Action` class initialized with the provided application object and action key.

    Examples
    --------
    >>> app = App()
    >>> action = create_action(app, 'some_action_key')
    >>> print(action)
    """
    return _Action(app, action_key, pset_key=pset_key)


In [18]:
#| export
@patch
def create_parameterset(app: App, action_key: str):
    """
    Creates and returns a parameter set for a specific action within the application.

    This function utilizes the `_action_info` dictionary to look up the parameter set information for the given action key.
    If the parameter set information exists for the specified action key, it creates and returns the corresponding parameter set object.

    Parameters
    ----------
    app : App
        The application object for which the parameter set is being created.
    action_key : str
        The key of the action for which the parameter set is being created.

    Returns
    -------
    HParameterSet object or None
        The parameter set object for the specified action key. 
        If there is no parameter set information for the given action key, None is returned.

    Examples
    --------
    >>> app = App()
    >>> params = create_parameterset(app, 'some_action_key')
    >>> print(params)
    """
    pset_key, description = _action_info.get(action_key, (None, None))
    if not pset_key:
        return None
    return getattr(app.api.HParameterSet, f"H{pset_key}")


In [19]:
#| hide
app = App()

In [20]:
#| export
@patch
def open(app: App, path: str):
    """
    Opens a file in the Hancom Office Hwp program using the provided file path.

    This function first converts the provided file path to an absolute path using the `get_absolute_path` function.
    It then opens the file in the Hancom Office Hwp program using the `api.Open` method and returns the absolute path of the opened file.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    path : str
        The file path of the document to be opened.

    Returns
    -------
    str
        The absolute path of the file that has been opened.

    Examples
    --------
    >>> app = App()
    >>> opened_file_path = open(app, 'path/to/document.hwp')
    >>> print(opened_file_path)
    """
    name = get_absolute_path(path)
    app.api.Open(name)
    return name


In [21]:
#| hide

app.open("test/공문양식.hwp")

'c:/Users/freed/Documents/python_projects/hwpapi/nbs/02_api/test/공문양식.hwp'

In [22]:
#| export
@patch
def get_hwnd(app: App):
    """
    Retrieves the window handle (HWND) of the active window in the Hancom Office Hwp program.

    This function accesses the active window in the Hancom Office Hwp program linked to the `App` instance
    and returns its window handle. The window handle can be used in scenarios where direct manipulation
    or interaction with the window at the OS level is required.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.

    Returns
    -------
    int
        The window handle (HWND) of the active Hancom Office Hwp window.

    Examples
    --------
    >>> app = App()
    >>> hwnd = get_hwnd(app)
    >>> print(hwnd)
    """
    return app.api.XHwpWindows.Active_XHwpWindow.WindowHandle


In [23]:
#| hide

app.get_hwnd()

199078

In [24]:
#| export
@patch
def save(app: App, path=None):
    """
    Saves the current document in the Hancom Office Hwp program.

    If a path is provided, the document is saved to that location in the appropriate format based on the file extension.
    Supported formats include HWP, PDF, HWPML2X (HWPX), and PNG. If no path is provided, the document is saved
    in its current location.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    path : str, optional
        The file path where the document will be saved. If None, the document is saved in its current location.

    Returns
    -------
    str
        The file path where the document has been saved.

    Examples
    --------
    >>> app = App()
    >>> saved_file_path = save(app, 'path/to/document.hwp')
    >>> print(saved_file_path)
    """
    if not path:
        app.api.Save()
        return app.get_filepath()
    
    name = get_absolute_path(path)
    extension = Path(name).suffix
    format_ = {
        ".hwp": "HWP",
        ".pdf": "PDF",
        ".hwpx": "HWPML2X",
        ".png": "PNG",
        ".txt": "TEXT", 
        ".docx": "MSWORD", 
    }.get(extension)

    app.api.SaveAs(name, format_)
    return name


In [25]:
#| hide

app.save("test.pdf")

'c:/Users/freed/Documents/python_projects/hwpapi/nbs/02_api/test.pdf'

In [26]:
#| export
@patch
def save_block(app: App, path: Path):
    """
    Saves a block of content in the Hancom Office Hwp program and returns the file path.

    This function saves a specified block from the Hancom Office Hwp program to a file in a given format.
    The format is determined by the file extension. Supported formats include HWP, PDF, HWPML2X (HWPX), and PNG.
    It returns the file path if the save operation is successful or None if it fails.

    Parameters
    ----------
    app : App
        The application object where the action will be performed.
    path : Path
        The file path where the block will be saved.

    Returns
    -------
    Path or None
        The path to the saved file if the save operation is successful, or None if it fails.

    Examples
    --------
    >>> app = App()
    >>> saved_path = save_block(app, Path('path/to/save/block.hwp'))
    >>> print(saved_path)
    """

    name = get_absolute_path(path)
    extension = Path(name).suffix
    format_ = {
        ".hwp": "HWP",
        ".pdf": "PDF",
        ".hwpx": "HWPML2X",
        ".png": "PNG",
    }.get(extension)

    action = app.actions.SaveBlockAction()
    p = action.pset
    
    p.filename = name
    p.Format = format_
    action.run()
    return name if Path(name).exists() else None


In [27]:
#| hide

app.save_block("test2.hwp")

In [28]:
#| export
@patch
def close(app: App):
    """
    Closes the currently open document in the Hancom Office Hwp program.

    This function triggers the 'FileClose' command within the Hancom Office Hwp program to close the current document.
    It's useful for programmatically managing documents within the application.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.

    Examples
    --------
    >>> app = App()
    >>> # Open and manipulate the document
    >>> close(app)
    """
    app.api.Run("FileClose")


In [29]:
#| hide

app.close()

In [30]:
#| export
@patch
def quit(app: App):
    """
    Terminates the Hancom Office Hwp program instance associated with the `App`.

    This function invokes the 'FileQuit' command within the Hancom Office Hwp program to close the application. 
    It's useful for programmatically controlling the lifecycle of the application.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.

    Examples
    --------
    >>> app = App()
    >>> # Perform actions with the app
    >>> quit(app)
    """
    app.api.Run("FileQuit")


In [31]:
#| export

@patch
def get_font_list(app:App):
    """
    Retrieves the list of fonts that used in the documnents from the current application.

    This method accesses the font list in the Hancom Office Hwp program linked to the `App` instance.
    It extracts fonts that used and return in list for further use or analysis.

    
    Parameters
    ----------
    app : App
        The application object from which the character shape settings will be retrieved.

    Returns
    -------
    List
        return font list that used in the hwp document.

    Examples
    --------
    >>> app = App()
    >>> font_list = app.get_font_list()
    >>> print(font_list)
    """
    app.api.ScanFont()
    result = app.api.GetFontList()
    output = list(map(lambda txt: txt.split(",")[0],
        result.split("\x02")
        ))
    return output

In [32]:
#| export
@patch
def get_charshape(app: App):
    """
    Retrieves the character shape settings from the current application.

    This method accesses the character shape settings in the Hancom Office Hwp program linked to the `App` instance.
    It extracts these settings and encapsulates them in a `CharShape` object for further use or analysis.

    Parameters
    ----------
    app : App
        The application object from which the character shape settings will be retrieved.

    Returns
    -------
    CharShape
        An object representing the character shape settings of the application.

    Examples
    --------
    >>> app = App()
    >>> char_shape = get_charshape(app)
    >>> print(char_shape)
    """
    action = app.actions.CharShape()
    p = action.pset
    return CharShape(p)


In [33]:
#| hide

app.get_charshape()

<CharShape: hangul_font=함초롬바탕, latin_font=함초롬바탕, text_color=0, fontsize=10.0, bold=0, italic=0, super_script=0, sub_script=0, offset=0, spacing=0, ratio=100, shade_color=#ffffff, shadow_color=#c0c0c0, shadow_offset_x=10, shadow_offset_y=10, strike_out_type=0, strike_out_color=0, underline_type=0, underline_shape=0, underline_color=0, out_line_type=0>

In [34]:
#| export
@patch
def set_charshape(app: App, charshape: CharShape=None, **kwargs):
    """
    Sets the character shape in the current paragraph of the Hancom Office Hwp application using the provided `CharShape`.

    If `charshape` is None, a default instance of `CharShape` is created. The function can also accept additional keyword 
    arguments (`kwargs`), which are assigned as properties of `charshape`. This modified `charshape` is then used to change 
    the current paragraph shape in the Hancom Office Hwp document.

    Parameters
    ----------
    app : App
        The `App` object. It is used to access the Hancom Office Hwp application.
    charshape : CharShape, optional
        The `CharShape` object to be used for setting character shapes in Hancom Office Hwp. The default is None.
    **kwargs
        Additional keyword arguments that are assigned as properties of `charshape`.

    Returns
    -------
    bool
        A boolean value indicating the success of the `set_charshape` operation.

    Examples
    --------
    >>> app = App()
    >>> char_shape = CharShape()
    >>> success = set_charshape(app, charshape=char_shape, fontName='Arial', fontSize=10)
    >>> print(success)
    """
    if charshape is None:
        charshape = CharShape()

    for key, value in kwargs.items():
        setattr(charshape, key, value)
        
    action = app.actions.CharShape()
    set_pset(action.pset, charshape.todict())
    return action.run()


In [35]:
#| hide

charshape = CharShape()
charshape.font = "바탕체"
charshape.super_script=1
charshape.sub_script = 1
app.set_charshape(charshape)

True

In [36]:
#| hide

app.get_charshape()

<CharShape: hangul_font=바탕체, latin_font=바탕체, text_color=0, fontsize=10.0, bold=0, italic=0, super_script=1, sub_script=1, offset=0, spacing=0, ratio=100, shade_color=#ffffff, shadow_color=#c0c0c0, shadow_offset_x=10, shadow_offset_y=10, strike_out_type=0, strike_out_color=0, underline_type=0, underline_shape=0, underline_color=0, out_line_type=0>

In [37]:
#| hide

charshape = CharShape()
charshape.font = "바탕체"
charshape.super_script=0
app.set_charshape(charshape)
charshape = app.get_charshape()

charshape.super_script == 0

True

In [38]:
#| export
@patch
def get_parashape(app: App):
    """
    Retrieves the paragraph shape settings from the current application.

    This method accesses the paragraph shape settings in the Hancom Office Hwp program linked to the `App` instance.
    It extracts these settings and encapsulates them in a `ParaShape` object for further use or analysis.

    Parameters
    ----------
    app : App
        The application object from which the paragraph shape settings will be retrieved.

    Returns
    -------
    ParaShape
        An object representing the paragraph shape settings of the application.

    Examples
    --------
    >>> app = App()
    >>> para_shape = get_parashape(app)
    >>> print(para_shape)
    """
    action = app.actions.ParagraphShape()
    p = action.pset

    return ParaShape(p)


In [39]:
#| hide

app.get_parashape()

<ParaShape: left_margin=0, right_margin=0, indentation=0, prev_spacing=0, next_spacing=0, line_spacing_type=Word, line_spacing=0, align_type=Both, break_latin_word=Word, break_non_latin_word=0, snap_to_grid=0, condense=0, widow_orphan=0, keep_with_next=0, page_break_before=0, text_alignment=Font, font_line_height=0, heading_type=None, level=0, border_connect=0, border_text=0, border_offset_left=0, border_offset_right=0, border_offset_top=0, border_offset_bottom=0, tail_type=0, line_wrap=0>

In [40]:
#| export

@patch
def set_parashape(app: App, parashape: ParaShape = None, **kwargs):
    """
    Sets the paragraph shape in the Hancom Office Hwp application using the provided `ParaShape`.

    If `parashape` is None, a default instance of `ParaShape` is created. The function can also accept additional keyword 
    arguments (`kwargs`), which are assigned as properties of `parashape`. This modified `parashape` is then used to change 
    the current paragraph shape in the Hancom Office Hwp document.

    Parameters
    ----------
    app : App
        The `App` object, used to access the Hancom Office Hwp application.
    parashape : ParaShape, optional
        The `ParaShape` object to be used for setting paragraph shapes in Hancom Office Hwp. Defaults to `None`.
    **kwargs
        Additional keyword arguments assigned as properties of `parashape`.

    Returns
    -------
    bool
        A boolean value indicating the success of the `set_parashape` operation.

    Examples
    --------
    >>> app = App()
    >>> para_shape = ParaShape()
    >>> success = set_parashape(app, parashape=para_shape, align='Left', indent=10)
    >>> print(success)
    """
    if parashape is None:
        parashape = ParaShape()

    for key, value in kwargs.items():
        setattr(parashape, key, value)

    action = app.actions.ParagraphShape()
    set_pset(action.pset, parashape.todict())
    return action.run()


In [41]:
#| hide

app.set_parashape(line_spacing=100)

True

In [42]:
#| export
@patch
def insert_text(
    app: App,
    text: str,
    charshape: CharShape = None, 
    **kwargs,
):
    """
    Inserts text into the Hancom Office Hwp document with specified character shape settings.

    This function inserts the given text into the document associated with the `App` instance. 
    It allows for optional character shape settings through a `CharShape` object. 
    Additional character shape attributes can be specified via keyword arguments.

    Parameters
    ----------
    app : App
        The `App` object associated with the Hancom Office Hwp program.
    text : str
        The text to be inserted into the document.
    charshape : CharShape, optional
        An optional `CharShape` object to specify the character shape settings for the inserted text. Defaults to `None`.
    **kwargs
        Additional character shape attributes to be set on the `charshape` object.

    Examples
    --------
    >>> app = App()
    >>> insert_text(app, "Hello World", fontName="Arial", fontSize=12)
    """
    if not charshape:
        charshape = CharShape()
    for key, value in kwargs.items():
        setattr(charshape, key, value)
    app.set_charshape(charshape)
    
    insert_text = app.actions.InsertText()
    p = insert_text.pset
    p.Text = text
    
    insert_text.run()
    return


In [43]:
#| hide

app.insert_text("안녕하세요", charshape)

In [44]:
#| hide

app.set_parashape(ParaShape(right_margin=20, align_type="Left"))

True

In [45]:
#| hide

parashape = app.actions.ParagraphShape()

In [46]:
#| hide

parashape.pset.RightMargin

0

In [47]:

#| export


def _get_text(app):
    """스캔한 텍스트 텍스트 제너레이터"""
    flag, text = 2, ""
    while flag not in [0, 1, 101, 102]:
        flag, text = app.api.GetText()
        yield text

@patch
@contextmanager
def scan(
    app: App,
    option=const.MaskOption.All,
    selection=False,
    scan_spos=const.ScanStartPosition.Document,
    scan_epos=const.ScanEndPosition.Document,
    spara=None,
    spos=None,
    epara=None,
    epos=None,
    scan_direction=const.ScanDirection.Forward,
):
    range_ = scan_spos.value + scan_epos.value
    if selection:
        range_ = 0x00FF  # Limit the scanning to the block if selection is True

    range_ += scan_direction.value
    app.api.InitScan(
        option=option.value,
        Range=range_,
        spara=spara,
        spos=spos,
        epara=epara,
        epos=epos,
    )
    yield _get_text(app)
    app.api.ReleaseScan()


In [48]:
#| export


def move_to_line(app: App, text):
    """인자로 전달한 텍스트가 있는 줄의 시작지점으로 이동합니다."""
    with app.scan(scan_spos="Line") as scan:
        for line in scan:
            if text in line:
                return app.move(key=const.MoveID.ScanPos)
    return False

In [49]:
#| export

@patch
def move(app: App, key=const.MoveId.ScanPos, para=None, pos=None):
    """
    Moves the caret position based on the specified key.

    Parameters
    ----------
    app : App
        The `App` object associated with the Hancom Office Hwp program.
    key : MoveId, optional
        The movement option, as defined in the `MoveId` Enum. Defaults to MoveId.ScanPos.
    para : int, optional
        The paragraph number to move to, if applicable. Defaults to None.
    pos : int, optional
        The position within the paragraph to move to, if applicable. Defaults to None.
    
    Returns
    -------
    bool
        True if the movement was successful, False otherwise.

    Examples
    --------
    >>> app = App()
    >>> move(app, key=MoveId.TopOfFile)
    """

    move_id = key.value if isinstance(key, const.MoveId) else const.MoveId[key].value
    return app.api.MovePos(moveID=move_id, Para=para, pos=pos)


In [50]:
#| export

@patch
def setup_page(
    app: App,  # app
    top=20,  # Top margin in millimeters
    bottom=10,  # Bottom margin in millimeters
    right=20,  # Right margin in millimeters
    left=20,  # Left margin in millimeters
    header=15,  # Header length in millimeters
    footer=5,  # Footer length in millimeters
    gutter=0,  # Gutter length (binding margin) in millimeters
):  
    """
    Sets up the page layout for the document in the Hancom Office Hwp application.

    This function configures the page margins, header, footer, and gutter (binding margin) sizes. 
    Measurements are in millimeters and are converted to the application's internal unit system.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    top : int, optional
        The top margin size in millimeters. Default is 20mm.
    bottom : int, optional
        The bottom margin size in millimeters. Default is 10mm.
    right : int, optional
        The right margin size in millimeters. Default is 20mm.
    left : int, optional
        The left margin size in millimeters. Default is 20mm.
    header : int, optional
        The header length in millimeters. Default is 15mm.
    footer : int, optional
        The footer length in millimeters. Default is 5mm.
    gutter : int, optional
        The gutter (binding margin) length in millimeters. Default is 0mm.

    Returns
    -------
    bool
        True if the page setup was successful, False otherwise.

    Examples
    --------
    >>> app = App()
    >>> setup_page(app, top=25, bottom=15, right=20, left=20, header=10, footer=5, gutter=5)
    """

    action = app.actions.PageSetup()
    p = action.pset

    p.PageDef.TopMargin = app.api.MiliToHwpUnit(top)
    p.PageDef.HeaderLen = app.api.MiliToHwpUnit(header)
    p.PageDef.RightMargin = app.api.MiliToHwpUnit(right)
    p.PageDef.BottomMargin = app.api.MiliToHwpUnit(bottom)
    p.PageDef.FooterLen = app.api.MiliToHwpUnit(footer)
    p.PageDef.LeftMargin = app.api.MiliToHwpUnit(left)
    p.PageDef.GutterLen = app.api.MiliToHwpUnit(gutter)

    return action.run()


In [51]:

#| export


@patch
def insert_picture(
    app: App,
    fpath,
    width=None,
    height=None,
    size_option=const.SizeOption.RealSize,
    reverse=False,
    watermark=False,
    effect=const.Effect.RealPicture,
):
    """
    Inserts a picture into the document with specified size and effect options.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    fpath : str
        File path of the picture to be inserted.
    width : int, optional
        Width of the picture, if None, determined by size option. Defaults to None.
    height : int, optional
        Height of the picture, if None, determined by size option. Defaults to None.
    size_option : SizeOption, optional
        The sizing option for the picture, defined in `SizeOption` Enum. Defaults to SizeOption.RealSize.
    reverse : bool, optional
        If true, the image is reversed. Defaults to False.
    watermark : bool, optional
        If true, the image is treated as a watermark. Defaults to False.
    effect : Effect, optional
        The visual effect for the picture, defined in `Effect` Enum. Defaults to Effect.RealPicture.

    Returns
    -------
    bool
        True if the picture was inserted successfully, False otherwise.

    Examples
    --------
    >>> app = App()
    >>> success = insert_picture(app, 'path/to/image.jpg', width=100, height=200, size_option=SizeOption.SpecificSize, effect=Effect.GrayScale)
    >>> print(success)
    """

    path = Path(fpath)
    size_option = size_option.value
    effect = effect.value

    return app.api.InsertPicture(
        path.absolute().as_posix(),
        Width=width,
        Height=height,
        sizeoption=size_option,
        reverse=reverse,
        watermark=watermark,
        effect=effect,
    )


In [52]:
#| export


@patch
def select_text(app: App, option=const.SelectionOption.Line):
    """
    Selects text in the document based on the specified option.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    option : SelectionOption, optional
        The text unit to be selected. Options are defined in SelectionOption Enum. Default is SelectionOption.Line.

    Returns
    -------
    tuple
        A tuple containing the results of the begin and end move actions (both boolean).

    Examples
    --------
    >>> app = App()
    >>> select_text(app, option=SelectionOption.Para)
    """

    begin_action_name, end_action_name = option.value
    begin_action = getattr(app.actions, begin_action_name)
    end_action = getattr(app.actions, end_action_name)

    return begin_action().run(), end_action().run()


In [53]:
#| hide


app.select_text()

(True, True)

In [54]:
#| hide


app.set_charshape(CharShape(font="바탕", fontsize=20))

True

In [55]:
#| export
@patch
def get_selected_text(app: App):
    """
    Retrieves text from the currently selected area in the Hancom Office Hwp document.

    This function scans the selected text in the document and returns it as a string.
    It's particularly useful for operations involving the currently highlighted or selected text segment.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.

    Returns
    -------
    str
        The text from the currently selected area in the document.

    Examples
    --------
    >>> app = App()
    >>> selected_text = get_selected_text(app)
    >>> print(selected_text)
    """

    with app.scan(selection=True) as scan:
        text = "\n".join(scan)
    return text


In [56]:
#| hide


app.get_selected_text()

'안녕하세요\n'

In [57]:
#| export 



# Refactored get_text function
@patch
def get_text(app: App, spos=const.ScanStartPosition.Line, epos=const.ScanEndPosition.Line):
    """
    Retrieves text from the document based on specified start and end positions.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    spos : ScanStartPosition, optional
        The start position for text retrieval. Default is ScanStartPosition.Line.
    epos : ScanEndPosition, optional
        The end position for text retrieval. Default is ScanEndPosition.Line.

    Returns
    -------
    str
        The retrieved text from the specified start to end positions.

    Examples
    --------
    >>> app = App()
    >>> text = get_text(app, spos=ScanStartPosition.Paragraph, epos=ScanEndPosition.Paragraph)
    >>> print(text)
    """

    with app.scan(scan_spos=spos, scan_epos=epos) as txts:
        text = "".join(txts)
    return text


In [58]:
#| hide


app.get_text()

'안녕하세요\r\n'

In [ ]:
#| export
@patch
def find_text(
    app: App,
    text="",  # 찾을 텍스트
    charshape: CharShape = CharShape(),  # 문자 모양 설정
    ignore_message=True,  # 메시지 무시 여부
    direction=const.Direction.Forward,  # 검색 방향
    match_case=False,  # 대소문자 구분
    all_word_forms=False,  # 모든 단어 형태 검색
    several_words=False,  # 여러 단어 검색
    use_wild_cards=False,  # 와일드카드 사용
    whole_word_only=False,  # 전체 단어만 검색
    replace_mode=False,  # 찾아 바꾸기 모드
    ignore_find_string=False,  # 찾을 문자열 무시
    ignore_replace_string=False,  # 바꿀 문자열 무시
    find_style="",  # 찾을 스타일
    replace_style="",  # 바꿀 스타일
    find_jaso=False,  # 자소로 검색
    find_reg_exp=False,  # 정규표현식으로 검색
    find_type=False,  # 마지막 검색(True), 새 검색(False)
):
    """
    문서에서 다양한 옵션을 사용해 특정 텍스트를 검색합니다.

    매개변수
    ----------
    app : App
        Hancom Office Hwp 프로그램과 연결된 `App` 인스턴스.
    text : str
        검색할 텍스트 문자열.
    [기타 매개변수...]

    반환값
    -------
    bool
        텍스트를 찾았으면 True, 그렇지 않으면 False.

    사용 예시
    --------
    >>> app = App()
    >>> found = find_text(app, text="Hello", direction=Direction.Forward)
    >>> print(found)
    """

    # 반복 검색 액션 생성
    action = app.actions.RepeatFind()
    p = action.pset
    p.FindCharShape = action.act.CreateSet()

    # 옵션 설정
    p.FindString = text
    p.IgnoreMessage = ignore_message
    p.MatchCase = match_case
    p.AllWordForms = all_word_forms
    p.Direction = direction.value  # Enum 값 사용
    p.SeveralWords = several_words
    p.UseWildCards = use_wild_cards
    p.WholeWordOnly = whole_word_only
    p.ReplaceMode = replace_mode
    p.IgnoreFindString = ignore_find_string
    p.IgnoreReplaceString = ignore_replace_string
    p.FindStyle = find_style
    p.ReplaceStyle = replace_style
    p.FindJaso = find_jaso
    p.FindRegExp = find_reg_exp
    p.FindType = find_type

    # 문자 모양 설정
    if charshape:
        set_pset(p.FindCharShape, charshape.todict())

    return action.run()


In [60]:
#| hide


app.find_text("안녕하세요")
app.find_text("안녕")
app.find_text("하세요")



False

In [61]:
app.find_text("안", charshape=CharShape(hangul_font="바탕"))
app.find_text("t", charshape=CharShape(hangul_font="가는안상수체", fontsize=21))


False

In [ ]:
#| export
@patch
def replace_all(
    app: App,
    old_text="",
    new_text="",
    old_charshape=CharShape(),  # 기존 문자 모양
    new_charshape=CharShape(),  # 새로운 문자 모양
    ignore_message=True,  # 메시지 무시 여부
    direction=const.Direction.All,  # 검색 방향
    match_case=False,  # 대소문자 구분
    all_word_forms=False,  # 모든 단어 형태
    several_words=False,  # 여러 단어 검색
    use_wild_cards=False,  # 와일드카드 사용
    whole_word_only=False,  # 전체 단어만 검색
    auto_spell=True,  # 자동 철자 교정
    replace_mode=True,  # 찾아 바꾸기 모드
    ignore_find_string=False,  # 찾을 문자열 무시
    ignore_replace_string=False,  # 바꿀 문자열 무시
    find_style="",  # 찾을 스타일
    replace_style="",  # 바꿀 스타일
    find_jaso=False,  # 자소로 검색
    find_reg_exp=False,  # 정규표현식으로 검색
    find_type=True,  # 마지막 검색 사용(True), 새 검색(False)
):
    """
    문서에서 특정 텍스트를 새 텍스트로 모두 교체합니다.

    매개변수
    ----------
    app : App
        Hancom Office Hwp 프로그램과 연결된 `App` 인스턴스.
    old_text : str
        교체할 텍스트 문자열.
    new_text : str
        대체할 새 텍스트 문자열.
    [기타 매개변수...]

    반환값
    -------
    bool
        교체 작업이 성공하면 True, 실패하면 False.

    사용 예시
    --------
    >>> app = App()
    >>> success = replace_all(app, old_text="Hello", new_text="Hi", direction=Direction.All)
    >>> print(success)
    """

    # 액션 생성
    action = app.actions.AllReplace(pset_key="FindReplace")
    p = action.pset

    # 옵션 설정
    p.FindString = old_text
    p.ReplaceString = new_text
    p.IgnoreMessage = ignore_message
    p.MatchCase = match_case
    p.AllWordForms = all_word_forms
    p.Direction = direction.value
    p.UseWildCards = use_wild_cards
    p.WholeWordOnly = whole_word_only
    p.AutoSpell = auto_spell
    p.ReplaceMode = replace_mode
    p.IgnoreFindString = ignore_find_string
    p.IgnoreReplaceString = ignore_replace_string
    p.FindStyle = find_style
    p.ReplaceStyle = replace_style
    p.FindJaso = find_jaso
    p.FindRegExp = find_reg_exp
    p.FindType = find_type

    # 기존 및 새로운 문자 모양 설정
    if old_charshape:
        set_pset(p.FindCharShape, old_charshape.todict())
    if new_charshape:
        set_pset(p.ReplaceCharShape, new_charshape.todict())

    return action.run()


In [63]:
#| hide


app.insert_text("t")

In [64]:
#| hide

app.replace_all("t", "txt", new_charshape=CharShape(hangul_font="가는안상수체", fontsize=21))

True

In [65]:
#| export
@patch
def insert_file(
    app: App,
    fpath,
    keep_charshape=False,
    keep_parashape=False,
    keep_section=False,
    keep_style=False,
):
    """
    Inserts the contents of a specified file into the current document.

    This function inserts the contents of another file into the current document at the cursor's position. 
    It provides options to retain various formatting attributes of the inserted content.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    fpath : str
        The file path of the document to be inserted.
    keep_charshape : bool, optional
        If True, retains the original character shapes from the inserted file. Defaults to False.
    keep_parashape : bool, optional
        If True, retains the original paragraph shapes from the inserted file. Defaults to False.
    keep_section : bool, optional
        If True, retains the original section formatting from the inserted file. Defaults to False.
    keep_style : bool, optional
        If True, retains the original styles from the inserted file. Defaults to False.

    Returns
    -------
    bool
        True if the file was inserted successfully, False otherwise.

    Examples
    --------
    >>> app = App()
    >>> success = insert_file(app, 'path/to/file.hwp', keep_style=True)
    >>> print(success)
    """

    action = app.actions.InsertFile()
    p = action.pset
    p.filename = Path(fpath).absolute().as_posix()
    p.KeepCharshape = keep_charshape
    p.KeepParashape = keep_parashape
    p.KeepSection = keep_section
    p.KeepStyle = keep_style

    return action.run()


In [66]:
#| export






@patch
def set_cell_border(
    app: App,
    top=None,
    right=None,
    left=None,
    bottom=None,
    horizontal=None,
    vertical=None,
    top_width=const.Thickness.NULL,
    right_width=const.Thickness.NULL,
    left_width=const.Thickness.NULL,
    bottom_width=const.Thickness.NULL,
    horizontal_width=const.Thickness.NULL,
    vertical_width=const.Thickness.NULL,
    top_color=None,
    bottom_color=None,
    left_color=None,
    right_color=None,
    horizontal_color=None,
    vertical_color=None,
):
    """
    Sets the border properties for cells in a table within a Hancom Office Hwp document.

    This function customizes the border types, widths, and colors for different sides of the cells. 
    It allows for detailed customization of cell appearance in tables.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    top, right, left, bottom, horizontal, vertical : int, optional
        Types of borders for the respective sides and internal lines of the cell (e.g., solid, dashed).
    top_width, right_width, left_width, bottom_width, horizontal_width, vertical_width : Thickness, optional
        Widths of the borders corresponding to the sides and internal lines of the cell. Specified using the `Thickness` Enum.
    top_color, bottom_color, left_color, right_color, horizontal_color, vertical_color : str, optional
        Colors for the respective borders in a string format (e.g., "#RRGGBB").

    Returns
    -------
    bool
        True if the border settings were successfully applied, False otherwise.

    Examples
    --------
    >>> app = App()
    >>> success = set_cell_border(app, top=1, bottom=1, top_width=Thickness._0_25_mm, bottom_width=Thickness._0_25_mm, top_color="#FF0000", bottom_color="#00FF00")
    >>> print(success)

    Notes
    -----
    The function relies on the `app.actions.CellFill` action to set the border properties. The `Thickness` Enum provides predefined thickness levels for the borders. The color parameters should be provided in hex format.
    """
    attrs = {
        "BorderTypeTop": top,
        "BorderTypeRight": right,
        "BorderTypeLeft": left,
        "BorderTypeBottom": bottom,
        "TypeHorz": horizontal,
        "TypeVert": vertical,
        "BorderWidthTop": top_width.value,
        "BorderWidthRight": right_width.value,
        "BorderWidthLeft": left_width.value,
        "BorderWidthBottom": bottom_width.value,
        "WidthHorz": horizontal_width.value,
        "WidthVert": vertical_width.value,
        "BorderColorTop": app.api.RGBColor(get_rgb_tuple(top_color))
        if top_color
        else None,
        "BorderColorRight": app.api.RGBColor(get_rgb_tuple(right_color))
        if right_color
        else None,
        "BorderColorLeft": app.api.RGBColor(get_rgb_tuple(left_color))
        if left_color
        else None,
        "BorderColorBottom": app.api.RGBColor(get_rgb_tuple(bottom_color))
        if bottom_color
        else None,
        "ColorHorz": app.api.RGBColor(get_rgb_tuple(horizontal_color))
        if horizontal_color
        else None,
        "ColorVert": app.api.RGBColor(get_rgb_tuple(vertical_color))
        if vertical_color
        else None,
    }

    action = app.actions.CellFill()
    p = action.pset

    for key, value in attrs.items():
        if value is None:
            continue
        setattr(p, key, value)

    return action.run()

In [67]:
#| export
@patch
def set_cell_color(
    app: App, bg_color=None, hatch_color="#000000", hatch_style=6, alpha=None
):
    """
    Sets the background color and hatch style for cells in a Hancom Office Hwp document.

    This function allows customization of the cell background, including color and hatching pattern,
    providing options to set transparency and hatch color.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    bg_color : str, optional
        Hexadecimal color code for the cell background (e.g., "#RRGGBB"). If not specified, the background is not changed.
    hatch_color : str, optional
        Hexadecimal color code for the hatching. Default is black ("#000000").
    hatch_style : int, optional
        Style of the hatching pattern. Default is 6.
    alpha : int, optional
        Alpha value for the background color's transparency (0-255). If not specified, transparency is not changed.

    Returns
    -------
    bool
        True if the cell color settings were applied successfully, False otherwise.

    Examples
    --------
    >>> app = App()
    >>> success = set_cell_color(app, bg_color="#FF0000", hatch_color="#0000FF", hatch_style=5, alpha=128)
    >>> print(success)

    Notes
    -----
    The function uses the `app.actions.CellBorderFill` action for setting the cell properties.
    Colors are specified in hexadecimal format. The alpha parameter controls the transparency of the background color.
    """

    fill_type = windows_brush = None
    if bg_color:
        fill_type = 1
        windows_brush = 1

    attrs = {
        "type": fill_type,
        "WindowsBrush": windows_brush,
        "WinBrushFaceColor": app.api.RGBColor(*get_rgb_tuple(bg_color)) if bg_color else None,
        "WinBrushAlpha": alpha,
        "WinBrushHatchColor": app.api.RGBColor(*get_rgb_tuple(hatch_color)) if hatch_color else None,
        "WinBrushFaceStyle": hatch_style,
    }

    action = app.actions.CellBorderFill()
    p = action.pset

    for key, value in attrs.items():
        if value is not None:
            setattr(p.FillAttr, key, value)

    return action.run()


In [68]:
#| hide


app = App()

In [69]:
#| hide


app.set_charshape(CharShape(hangul_font="문화바탕", font_type=2))

True

In [70]:
#| hide


action = app.actions.CharShape()

In [71]:
#| hide


app.get_charshape()

<CharShape: hangul_font=문화바탕, latin_font=바탕, text_color=0, fontsize=21.0, bold=0, italic=0, super_script=0, sub_script=1, offset=0, spacing=0, ratio=100, shade_color=#ffffff, shadow_color=#c0c0c0, shadow_offset_x=10, shadow_offset_y=10, strike_out_type=0, strike_out_color=0, underline_type=0, underline_shape=0, underline_color=0, out_line_type=0>

In [72]:
#| hide


app.api.HwpLineWidth("1.0mm")

10

In [73]:
#| hide

app.set_cell_border(bottom=3)

False

In [74]:
#| hide

get_rgb_tuple("#123456")

(18, 52, 86)

In [75]:
#| hide

app.set_cell_color(bg_color="#123456")

False

In [76]:
#| hide

# app.api.CurSelectedCtrl.CtrlID

In [77]:
#| hide

# 동작은 되나 너무 느리다. 
# 빠르게 할 수 있는 방법을 찾아보자. 
# 양식을 설정할 수 있는 방법을 생각해 보자.
# 어떻게 해야 위아래로 값이 같은 셀을 병합할 수 있을지 생각해 보자.
# block_input을 해야 그나마 쓸 수 있을 듯

def insert_dataframe(app, df):
    action = app.actions.TableCreate()
    p = action.pset
    p.WidthType = 1    # fit size
    p.Rows = df.shape[0]+1  # add header
    p.Cols = df.shape[1]
    action.run()

    for col in df.columns:
        app.insert_text(col)
        app.move("RightOfCell")
    for i, row in df.iterrows():
        for item in row.values:
            app.insert_text(item)
            app.move("RightOfCell")
    return app.api.ParentCtrl

In [78]:
#| hide


## 더 빠르고 안정적일 듯
# 초단위로 걸려서 block_input을 해야 할 듯 
# value 중에서 tab이 있으면 replace는 해야 할 듯

def convert_to_text(value):
    value = str(value).replace("\t", "    ")
    return value

def insert_dataframe(app:App, df):
    start_pos = app.api.GetPos()
    text = ""
    text += "\t".join(map(convert_to_text, df.columns))
    for i, row in df.iterrows():
        text += "\r\n"
        text += "\t".join(map(convert_to_text, row.values))
    app.insert_text(text)
    end_pos = app.api.GetPos()

    app.api.SelectText(start_pos[1], start_pos[2], end_pos[1], end_pos[2])

    return app.actions.TableStringToTable().run()

In [122]:
#| hide

#@property
@patch(as_prop=True)
def page_width(app:App):
    action = app.actions.PageSetup()
    pset = action.create_pset()
    paper_width = pset.Item("PageDef").Item("PaperWidth")
    left_margin = pset.Item("PageDef").Item("LeftMargin")
    right_margin = pset.Item("PageDef").Item("RightMargin")
    return unit2mili(paper_width - (left_margin + right_margin))

@patch(as_prop=True)
def page_height(app:App):
    action = app.actions.PageSetup()
    pset = action.create_pset()
    paper_height = pset.Item("PageDef").Item("PaperHeight")
    top_margin = pset.Item("PageDef").Item("TopMargin")
    bottom_margin = pset.Item("PageDef").Item("BottomMargin")
    return unit2mili(paper_height - (top_margin + bottom_margin))

In [123]:
app = App()
app.page_height

262.42402826855124

In [120]:
#| export
def get_cell_property(app:App):
    action = app.actions.TablePropertyDialog()
    pset = action.create_pset()
    cell = pset.Item("ShapeTableCell")
    property_names = ("HasMargin", "Protected", "Header", "Width", "Height", "Editable", "Dirty", "CellCtrlData")
    return {name: cell.Item(name) for name in property_names} if cell else {}

def get_shape_property(app:App):
    property_names = ("TreatAsChar", "AffectsLine", "VertRelTo", "VertAlign", "VertOffset", "HorzRelTo", "HorzAlign", "HorzOffset", "FlowWithText", "AllowOverlap", "WidthRelTo", "Width", "HeightRelTo", "Height", "ProtectSize", "TextWrap", "TextFlow", "OutsideMarginLeft", "OutsideMarginRight", "OutsideMarginTop", "OutsideMarginBottom", "NumberingType", "LayoutWidth", "LayoutHeight", "Lock", "HoldAnchorObj", "PageNumber", "AdjustSelection", "AdjustTextBox", "AdjustPrevObjAttr")
    action = app.actions.TablePropertyDialog()
    pset = action.create_pset()
    return {name: pset.Item(name) for name in property_names}

@patch(as_prop=True)
def table_width(app:App):
    table_property = get_shape_property(app)
    return unit2mili(table_property.get("Width"))

@patch(as_prop=True)
def table_height(app:App):
    table_property = get_shape_property(app)
    return unit2mili(table_property.get("Height"))

@patch(as_prop=True)
def cell_width(app:App):
    cell_property = get_cell_property(app)
    return unit2mili(cell_property.get("Width"))

@patch(as_prop=True)
def cell_height(app:App):
    cell_property = get_cell_property(app)
    return unit2mili(cell_property.get("Height"))

@patch
def set_cell_width(app:App, width):
    action = app.actions.TablePropertyDialog()
    action.pset.ShapeTableCell.Width = mili2unit(width)
    action.run()
    print(app.cell_width, width)
    return app.cell_width == width

@patch
def set_cell_height(app:App, height):
    action = app.actions.TablePropertyDialog()
    action.pset.ShapeTableCell.Height = mili2unit(height)
    action.run()
    return app.cell_height == height

In [82]:
#| hide

import pandas as pd
import numpy as np



In [83]:
#| hide

df = pd.DataFrame(np.random.random(100).reshape(10, 10))
dir(app.api)

['Application',
 'ArcType',
 'AutoNumType',
 'BorderShape',
 'BreakWordLatin',
 'BrushType',
 'CLSID',
 'Canonical',
 'CellApply',
 'CellShape',
 'CharShadowType',
 'CharShape',
 'CheckXObject',
 'Clear',
 'ColDefType',
 'ColLayoutType',
 'ConvertPUAHangulToUnicode',
 'CreateAction',
 'CreateField',
 'CreateID',
 'CreateMode',
 'CreatePageImage',
 'CreateSet',
 'CrookedSlash',
 'CurFieldState',
 'CurMetatagState',
 'CurSelectedCtrl',
 'DSMark',
 'DbfCodeType',
 'DeleteCtrl',
 'Delimiter',
 'DrawAspect',
 'DrawFillImage',
 'DrawShadowType',
 'EditMode',
 'Encrypt',
 'EndSize',
 'EndStyle',
 'EngineProperties',
 'ExportStyle',
 'FieldExist',
 'FileTranslate',
 'FillAreaType',
 'FindCtrl',
 'FindDir',
 'FindPrivateInfo',
 'FontType',
 'GetBinDataPath',
 'GetCurFieldName',
 'GetCurMetatagName',
 'GetFieldList',
 'GetFieldText',
 'GetFileInfo',
 'GetFontList',
 'GetHeadingString',
 'GetMessageBoxMode',
 'GetMetatagList',
 'GetMetatagNameText',
 'GetMousePos',
 'GetPageText',
 'GetPos',
 'Ge

#| hide

## 

cell을 순환하면서 모양을 잡을 수 있을 것으로 예상된다.
6개 값을 받아서 정렬하면 되지 않을까 싶다.


In [84]:
#| hide

insert_dataframe(app, df)
app.api.FindCtrl()
app.actions.ShapeObjTextBoxEdit().run()
app.set_cell_border(False, False)
for _ in range(10):
    app.actions.TableCellBlockRow().run()
    app.set_cell_color(bg_color=(253, 123, 253), hatch_style=6)
    app.set_cell_border(False, False, False, False)

    app.actions.TableCellBlock().run()
    app.move("DownOfCell")

In [85]:

import re


TOKENS = [
    ('HEADER1', r'(^|\n)# (.*?)(\n|$)'),
    ('HEADER2', r'(^|\n)## (.*?)(\n|$)'),
    ('HEADER3', r'(^|\n)### (.*?)(\n|$)'),
    ('HEADER4', r'(^|\n)#### (.*?)(\n|$)'),
    ('HEADER5', r'(^|\n)##### (.*?)(\n|$)'),
    ('HEADER6', r'(^|\n)###### (.*?)(\n|$)'),
    ('BOLD', r'\*\*(.*?)\*\*'),
    ('ITALIC', r'\*(.*?)\*'),
    ('LIST_ITEM', r'(^|\n)( *)(-|\+|\*) (.*?)(\n|$)'),
    ('ORDERED_LIST_ITEM', r'(^|\n)( *)(\d+)\. (.*?)(\n|$)'),
    ('TABLE_ROW', r'(^|\n)\|.*?\|(\n|$)'),
    ('TABLE_SEPARATOR', r'(^|\n)\|[-\s|]*?\|(\n|$)'),
    ('NEWLINE', r'\n'),
    ('TEXT', r'[^#\*\-\n\|]+')
]


import re

def tokenize(text):
    tokens = []
    while text:
        for token_type, pattern in TOKENS:
            match = re.match(pattern, text)
            if match:
                if token_type == 'LIST_ITEM' or token_type == 'ORDERED_LIST_ITEM':
                    indent = len(match.group(2))
                    tokens.append(('LIST_ITEM', indent, match.group(4).strip()))
                else:
                    tokens.append((token_type, match.group().strip()))
                text = text[match.end():]
                break
        else:
            # 매칭되는 패턴이 없는 경우 텍스트에서 첫 글자를 잘라냄
            text = text[1:]
    return tokens

markdown_text = """
# Hello World

This is a **bold** text and this is *italic* text.

- Item 1
  - Subitem 1.1
  - Subitem 1.2
    - Subsubitem 1.2.1
- Item 2
"""

tokens = tokenize(markdown_text)
for token in tokens:
    print(token)



('HEADER1', '# Hello World')
('NEWLINE', '')
('TEXT', 'This is a')
('BOLD', '**bold**')
('TEXT', 'text and this is')
('ITALIC', '*italic*')
('TEXT', 'text.')
('NEWLINE', '')
('LIST_ITEM', 0, 'Item 1')
('LIST_ITEM', 2, 'Subitem 1.1')
('LIST_ITEM', 2, 'Subitem 1.2')
('LIST_ITEM', 4, 'Subsubitem 1.2.1')
('LIST_ITEM', 0, 'Item 2')


In [86]:
#| hide
import nbdev

nbdev.nbdev_export()